# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer - Nanodegree/0 - Data Modeling/Project 2
['/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer - Nanodegree/0 - Data Modeling/Project 2/event_data/2018-11-15-events.csv', '/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer - Nanodegree/0 - Data Modeling/Project 2/event_data/2018-11-22-events.csv', '/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer - Nanodegree/0 - Data Modeling/Project 2/event_data/2018-11-09-events.csv', '/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer - Nanodegree/0 - Data Modeling/Project 2/event_data/2018-11-18-events.csv', '/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer - Nanodegree/0 - Data Modeling/Project 2/event_data/2018-11-04-events.csv', '/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer - Nanodegree/0 - Data Modeling/Project 2/event_data/2018-11-01-events.csv', '/Users/erich/Documents/Personal/Courses/Udacity/Data Engineer

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in the new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part 2 - Generating Keyspace and modeling tables

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


## Queries list

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### 1st Table creation and data insertion

#### sessions_history table
For this table, the columns "session_id" and "item_in_session", were chosen to be part of a composite key because the query uses "user_id" and "session_id" as filters in the query "where" clause.

In [42]:
## Create statement to generate the table for the respective query.

query = "create table if not exists sessions_history "
query = query + "(session_id int, item_in_session int, artist text, song text, song_length float, primary key(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [43]:
## Loading data from the .csv file to the table.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into sessions_history (session_id, item_in_session, artist, song, song_length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### 1st Query

In [44]:
## Select statement to get the 1st query results: Artist Name, Song and Song Length by a given SessionId and ItemInSession.
query1 = """
    select\
        artist\
        ,song\
        ,song_length\
    from sessions_history\
    where\
        session_id = 338\
    and item_in_session = 4\
        """

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

lst = []

for row in rows:
    lst.extend([[row.artist, row.song, row.song_length]])

pd.DataFrame(lst, columns = ['artist', 'song', 'song_length'])

,artist,song,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### 2nd Table creation and data insertion

#### users_activity table
For this table, the columns "user_id" and "session_id", were chosen to be part of a composite partition key because of two main points:
- the query uses "user_id" and "session_id" as filters in the query "where" clause;
- the data in the table will be distributed by session and users in different nodes on Cassandra, which improves the overall query performance.

The "item_in_session", is being used as a clustering key not only to make a better unique key, it will sort the data while querying (it will sort the "song" column).

In [37]:
## Create statement to generate the table for the respective query.

query = "create table if not exists users_activity "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text,\
                     user_first_name text, user_last_name text, PRIMARY KEY ( (user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [38]:
## Loading data from the .csv file to the table.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into users_activity (user_id, session_id, item_in_session, artist, song, user_first_name, user_last_name)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### 2nd Query

In [40]:
## Select statement to get the 2nd query results: Artist Name, Song Name, User First and Last Names by a given userId and SessionId.

query2 = """
    select\
        artist\
        ,song\
        ,user_first_name\
        ,user_last_name\
    from users_activity\
    where\
        user_id = 10\
    and session_id = 182\
        """

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

lst = []

for row in rows:
    lst.extend([[row.artist, row.song, row.user_first_name, row.user_last_name]])

pd.DataFrame(lst, columns = ['artist', 'song', 'user_first_name', 'user_last_name'])


,artist,song,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3rd Table creation and data insertion

#### song_history table
For this table, the columns "song" and "user_id", were chosen to be part of a composite key because they are used as filters in the query "where" clause;

In [45]:
## Create statement to generate the table for the respective query.

query = "create table if not exists song_history "
query = query + "(song text, user_id int, user_first_name text, user_last_name text, primary key(song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [47]:
## Loading data from the .csv file to the table.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into song_history (song, user_id, user_first_name, user_last_name)"
        query = query + "values (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### 3rd Query

In [67]:
## Select statement to get the 3rd query results: User First and Last Names by a given song

query2 = """
    select\
        user_first_name\
        ,user_last_name\
    from song_history\
    where\
        song = 'All Hands Against His Own' \
        """

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

lst = []

for row in rows:
    lst.extend([[row.user_first_name, row.user_last_name]])

pd.DataFrame(lst, columns = ['user_first_name', 'user_last_name'])

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Tables cleansing

In [68]:
session.execute("drop table if exists song_history")
session.execute("drop table if exists users_activity")
session.execute("drop table if exists sessions_history")

### Close the session and cluster connection¶

In [69]:
session.shutdown()
cluster.shutdown()